In [1]:
import pandas as pd
import numpy as np

cols = ['id', 'tipodepropiedad', 'metroscubiertos', 'metrostotales']

types = {'id': int, 'tipodepropiedad': str, 'metroscubiertos': float, 'metrostotales': float}

training_data = pd.read_csv('../../data/TP2/train.csv', usecols = cols, dtype = types)
evaluation_data = pd.read_csv('../../data/TP2/test.csv', usecols = cols, dtype = types)

training_data.head()

,id,tipodepropiedad,metroscubiertos,metrostotales
0,254099,Apartamento,80.0,80.0
1,53461,Casa en condominio,268.0,180.0
2,247984,Casa,144.0,166.0
3,209067,Casa,63.0,67.0
4,185997,Apartamento,95.0,95.0


In [2]:
training_data['metroscubiertos'] = training_data['metroscubiertos'].apply(lambda x: 50 if np.isnan(x) else x)
evaluation_data['metroscubiertos'] = evaluation_data['metroscubiertos'].apply(lambda x: 50 if np.isnan(x) else x)

training_data['tipodepropiedad'] = training_data['tipodepropiedad'].apply(lambda x: str(x))
evaluation_data['tipodepropiedad'] = evaluation_data['tipodepropiedad'].apply(lambda x: str(x))

max_metros_by_tipodepropiedad = training_data.groupby(['tipodepropiedad']).agg({'metroscubiertos':'max', 'metrostotales':'max'}).reset_index()
min_metros_by_tipodepropiedad = training_data.groupby(['tipodepropiedad']).agg({'metroscubiertos':'min', 'metrostotales':'min'}).reset_index()

metroscubiertos_maxes = {}
for tipo in max_metros_by_tipodepropiedad['tipodepropiedad'].values:
    metroscubiertos_maxes[tipo] = max_metros_by_tipodepropiedad.loc[max_metros_by_tipodepropiedad.tipodepropiedad == tipo].metroscubiertos.values[0]
    
metroscubiertos_mins = {}
for tipo in max_metros_by_tipodepropiedad['tipodepropiedad'].values:
    metroscubiertos_mins[tipo] = min_metros_by_tipodepropiedad.loc[min_metros_by_tipodepropiedad.tipodepropiedad == tipo].metroscubiertos.values[0]
    
metrostotales_maxes = {}
for tipo in max_metros_by_tipodepropiedad['tipodepropiedad'].values:
    metrostotales_maxes[tipo] = max_metros_by_tipodepropiedad.loc[max_metros_by_tipodepropiedad.tipodepropiedad == tipo].metrostotales.values[0]
    
metrostotales_mins = {}
for tipo in max_metros_by_tipodepropiedad['tipodepropiedad'].values:
    metrostotales_mins[tipo] = min_metros_by_tipodepropiedad.loc[min_metros_by_tipodepropiedad.tipodepropiedad == tipo].metrostotales.values[0]
    
metrostotales_mins

{'Apartamento': 15.0,
 'Bodega comercial': 21.0,
 'Casa': 15.0,
 'Casa en condominio': 15.0,
 'Casa uso de suelo': 34.0,
 'Departamento Compartido': 20.0,
 'Duplex': 34.0,
 'Edificio': 20.0,
 'Garage': nan,
 'Hospedaje': nan,
 'Huerta': 27.0,
 'Inmuebles productivos urbanos': 22.0,
 'Local Comercial': 15.0,
 'Local en centro comercial': 19.0,
 'Lote': 161.0,
 'Nave industrial': 61.0,
 'Oficina comercial': 15.0,
 'Otros': 27.0,
 'Quinta Vacacional': 15.0,
 'Rancho': 20.0,
 'Terreno': 15.0,
 'Terreno comercial': 17.0,
 'Terreno industrial': 118.0,
 'Villa': 43.0,
 'nan': 35.0}

In [3]:
def minmax_scale_metroscubiertos(row):
    tipo = row.tipodepropiedad
    return ((row['metroscubiertos'] - metroscubiertos_mins[tipo]) / (metroscubiertos_maxes[tipo] - metroscubiertos_mins[tipo]))

def minmax_scale_metrostotales(row):
    tipo = row.tipodepropiedad
    return ((row['metrostotales'] - metrostotales_mins[tipo]) / (metrostotales_maxes[tipo] - metrostotales_mins[tipo]))

training_data['minmaxed_scaled_metroscubiertos_by_tipodepropiedad'] = training_data.apply(minmax_scale_metroscubiertos, axis = 1)
evaluation_data['minmaxed_scaled_metroscubiertos_by_tipodepropiedad'] = evaluation_data.apply(minmax_scale_metroscubiertos, axis = 1)

training_data['minmaxed_scaled_metrostotales_by_tipodepropiedad'] = training_data.apply(minmax_scale_metrostotales, axis = 1)
evaluation_data['minmaxed_scaled_metrostotales_by_tipodepropiedad'] = evaluation_data.apply(minmax_scale_metrostotales, axis = 1)

training_data.drop(columns=['tipodepropiedad', 'metroscubiertos'], inplace = True)
evaluation_data.drop(columns=['tipodepropiedad', 'metroscubiertos'], inplace = True)

evaluation_data.head()

/home/pablo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/home/pablo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
/home/pablo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys


,id,metrostotales,minmaxed_scaled_metroscubiertos_by_tipodepropiedad,minmaxed_scaled_metrostotales_by_tipodepropiedad
0,4941,NaN,0.672170,NaN
1,51775,67.0,0.120853,0.122931
2,115253,100.0,0.168246,0.200946
3,299321,86.0,0.165877,0.167849
4,173570,76.0,0.153302,0.143868


In [4]:
training_data.to_csv('../../res/ftr/minmax_scaled_surface_by_tipodepropiedad_train.csv')
evaluation_data.to_csv('../../res/ftr/minmax_scaled_surface_by_tipodepropiedad_evaluation.csv')